Сначала импортируем все, что нам понадобится. И загрузим набор данных MNIST:

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Как мы уже обсуждали, датасет состоит из 70 000 размеченных черно-белых изображений, которые разделены между тренировочной, тестовой и валидационной выборками по 55 000,10 000 и 5000 примеров соответственно.   

Зададим гиперпараметры сети:

In [2]:
batch_size = 64
latent_space = 128
learning_rate = 0.1

Давайте для простоты эксперимента опишем однослойный автокодировщик. Для этого объявим его веса:

In [4]:
ae_weights = {"encoder_w": tf.Variable(tf.truncated_normal([784, latent_space], stddev=0.1)),
              "encoder_b": tf.Variable(tf.truncated_normal([latent_space], stddev=0.1)),
              "decoder_w": tf.Variable(tf.truncated_normal([latent_space, 784], stddev=0.1)), 
              "decoder_b": tf.Variable(tf.truncated_normal([784], stddev=0.1))}

И зададим тензоры:

In [5]:
ae_input = tf.placeholder(tf.float32, [batch_size, 784])
hidden = tf.nn.sigmoid(tf.matmul(ae_input, ae_weights["encoder_w"]) + ae_weights["encoder_b"])
visible_logits = tf.matmul(hidden, ae_weights["decoder_w"]) + ae_weights["decoder_b"]
visible = tf.nn.sigmoid(visible_logits)

На самом деле тензор visible для обучения не нужен, так как мы будем использовать стандартную функцию ошибки из библиотеки TensorFlow, sigmoid_cross_entropy_with_logits. Однако позже он пригодится нам для того, чтобы визуализировать восстановленные изображения. А сама функция ошибки для
автокодировщика в виде перекрестной энтропии запишется так:

In [9]:
ae_cost = tf.reduce_mean(
    tf.nn.sigmoid_cross_entropy_with_logits(logits=visible_logits, labels=ae_input))

Выберем алгоритм оптимизации из TensorFlow, в данном случае AdaGrad:

In [10]:
optimizer = tf.train.AdagradOptimizer(learning_rate)
ae_op = optimizer.minimize(ae_cost)

Как и раньше, нам остается только инициализировать переменные, создать сессию и запустить процесс обучения:

In [11]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

for i in range(10000):
    x_batch, _ = mnist.train.next_batch(batch_size)
    sess.run(ae_op, feed_dict={ae_input: x_batch})

После обучения на 10000 мини-батчей ошибка восстановления на тестовых данных составляет примерно 0,22, а после 100000 падает до 0,12 и при дальнейшем обучении может быть улучшена до значений, меньших чем 0,1. Изображения, которые получаются после восстановления, все еще выглядят как цифры, но очень сильно «размыты».   

Чтобы сделать разреженный автокодировщик, нужно, как мы обсуждали выше, добавить регуляризатор. Зададим в качестве дополнительных параметров rho и beta вес регуляризации в функции стоимости:

In [12]:
rho = 0.05
beta = 1.0

Определим теперь тензор для регуляризационного слагаемого:

In [13]:
data_rho = tf.reduce_mean(hidden, 0)
reg_cost = - tf.reduce_mean(tf.log(data_rho/rho) * rho + tf.log((1-data_rho)/(1-rho)) * (1-rho))

Здесь для оценки p^ мы используем среднее значение активации скрытого слоя по мини-батчу для каждого нейрона, а для упрощения вычислений переворачиваем дроби внутри логарифмов. Общая функция стоимости теперь выглядит так:

In [14]:
total_cost = ae_cost + beta * reg_cost

И именно она передается оптимизатору вместо ae_cost.   

Посмотрите внимательно, что здесь произошло: мы задали дополнительное слагаемое для функции ошибки; оно тем меньше, чем ближе распределение активации нейронов к придуманному нами распределению, в котором монетка выпадает орлом (то есть нейрон активируется) с вероятностью р = 0.05. Иными словами,
мы просто попросили модель обучиться так, чтобы на каждом отдельном входе на скрытом слое активировалось примерно 5 % нейронов; а параметр beta отвечает за то, насколько убедительно мы ее об этом попросили.   

После нескольких миллионов мини-батчей и нескольких эпох обучения модели средние значения нейронов скрытого слоя уменьшатся и станут близки к р.

Но где же обещанная разреженность? Ведь хоть значения активаций и уменьшились, они все так же остаются ненулевыми, а нам для улучшения эффективности модели неинтересно иметь околонулевые веса, нам нужны строгие, «жесткие» нули. Давайте проведем небольшой эксперимент: возьмем случайный мини-батч, спроецируем его в вектор активаций скрытого слоя и обнулим все элементы, меньшие 0,1. Поскольку в наших экспериментах р = 0,05, это весьма значительная часть элементов скрытого слоя. Это само по себе вполне ожидаемо, но самое интересное здесь то, что оставшихся элементов будет вполне достаточно для восстановления исходных изображений! Разреженный автокодировщик не обманул: мы действительно обучили скрытый слой так, что теперь можно обнулить все маленькие веса и все еще получить хорошее приближение к данным.   

Чтобы это нарисовать, введем дополнительный «зашумленный скрытый слой» и его декодировщик:

In [15]:
noised_hidden = tf.nn.relu(hidden - 0.1) + 0.1
noised_visible = tf.nn.sigmoid(
    tf.matmul(noised_hidden, ae_weights["decoder_w"]) + ae_weights["decoder_b"])

Для того чтобы реализовать шумоподавляющий автокодировщик и протестировать его на MNIST, достаточно внести минимальные изменения в уже написанный код. Начнем с того, что добавим еще одну заглушку для «испорченных» данных:

In [16]:
noisy_input = tf.placeholder(tf.float32, [batch_size, 784])

Конечно, эта заглушка должна иметь такой же размер, как и ae_input, так как функция потерь после сжатия и восстановления примеров будет подсчитываться на основе исходных данных, а не зашумленных. Скрытый слой теперь тоже формируется с использованием новой заглушки:

In [17]:
hidden = tf.nn.sigmoid(tf.matmul(noisy_input, ae_weights["encoder_w"]) + ae_weights["encoder_b"])

А когда нам нужно будет для тестирования полученного автокодировщика прогнать через скрытый слой «чистые» данные, без шума, мы просто передадим исходный батч в noisy_input. Основной цикл обучения меняется незначительно:

In [ ]:
noise_prob = 0.3
updates = 10000

for i in range(updates):
    x_batch, _ = mnist.train.next_batch(batch_size)
    noise_mask = np.random.uniform(0., 1., [batch_size, 784]) < noise_prob
    noisy_batch = x_batch.copy()
    noisy_batch[noise_mask] = 0.0
    sess.run(ae_op, feed_dict={ae_input: x_batch, noisy_input: noisy_batch})

Функция np.random.uniform возвращает массив со значениями, выбранными (сэмплированными) из случайной величины, равномерно распределенной на отрезке, который задается первыми двумя параметрами; в нашем случае это отрезок [0,1]. Третий параметр задает размер массива. Обратите внимание на то, что перед
обнулением элементов входных данных мы копируем батч целиком, вызывая x_batch.copy(); если бы мы этого не сделали, а использовали обычное присваивание, то в результате обнулились бы значения и в x_batch. Для эксперимента мы выбрали вероятность «выкидывания» пиксела noise_prob = 0.3. Эта вероятность на первый взгляд кажется очень большой: как так, мы выкидываем 30 % картинки? Да что там вообще останется? Оказывается, что на практике наилучшие результаты получаются как раз в случаях, когда уровень шума очень, на первый взгляд чрезмерно, большой: просить модель угадывать «закрытую» треть, а то и половину картинки оказывается правильной стратегией.